# LAB 2 &ndash; Shuffling cards

In this second lab, you'll get acquainted with AES and learn how to cryptographically shuffle a deck of cards.

First thing to do: double-click on this text cell to write your <b style="color: red">HACKER TEAM NAME HERE</b>.

## 0) Bytes

Byte (« octets ») arrays in Python 3 can be thought of a list of integers between 0 and 255: 

In [10]:
a = bytes([72, 69, 76, 76, 79])

a

b'HELLO'

The corresponding ASCII characters are displayed for convenience (the `b` reminds us that it is not a regular string but a byte array), but the individual bytes should not be thought of as "characters" (integers are returned):

In [2]:
for c in a:
    print(c)

72
69
76
76
79


Also, be aware that most values don't correspond to printable ASCII characters and just won't display nicely:

In [15]:
b = bytes([145, 8, 203, 78, 23])
b

b'\x91\x08\xcbN\x17'

Hexadecimal values are displayed for bytes for which there is no better option; in general, this is the preferred way to look at a byte array (2 hexadecimal numbers for each byte).

In [16]:
b.hex()

'9108cb4e17'

It's easy to implement the XOR (or $\oplus$) operation for bytes since Python has a built-in bitwise-XOR operator for integers.

In [19]:
def xor(a: bytes, b: bytes):

    return bytes([x^y for x,y in zip(a,b)])

print("a      :", a.hex())
print("b      :", b.hex())
print("a xor b:", xor(a,b).hex())

a      : 48454c4c4f
b      : 9108cb4e17
a xor b: d94d870258


**To do:** Before you proceed, make sure everything above makes sense to you (experience shows that you will probably need to read it again in a couple of minutes)...

As a warm-up, make sure you are able to turn a string of hex digits (such as `'48454c4c4f'`) into a `bytes` object (such as `a`).

In [20]:
c = "48454c4c4f"
bytes.fromhex(c)

b'HELLO'

## 1) Using AES

We will use the implementation of AES provided by the <a href="https://pypi.org/project/cryptography/">cryptography</a> library (`pip install cryptography` if needed).

In [9]:
import os

# Hazardous Materials: use in real-world applications only if you know what you're doing!
# cryptography also provides "idiot-proof" Recipes that should be preferred

from cryptography.hazmat.primitives.ciphers import Cipher
from cryptography.hazmat.primitives.ciphers.algorithms import AES
from cryptography.hazmat.primitives.ciphers.modes import ECB
from cryptography.hazmat.backends import default_backend

AES works with blocks of 128 bits, _i.e._ 16 bytes.

In [ ]:
AES.block_size # in bits

So let us now initialize AES with a randomly generated 16-byte key. 

In [23]:
k = os.urandom(16)

print("Secret key:", k.hex())

cipher = Cipher(AES(k), ECB(), default_backend())  # E(k, ) in the slides

encryptor = cipher.encryptor()

Secret key: e9a9de8e7a3d10d7b4bee8bc114e5f88


We can now start encrypting text. 

In [62]:
m = b"Don't forget to respect the block length. A reversible padding scheme is needed in general.#####"

encryptor = cipher.encryptor()
c = encryptor.update(m)

for i in range(len(c)//16):
    print(c[16*i:16*(i+1)].hex())

9b1534630e09279bebcd1776fc61de86
4fb5f143f26830b370ca7cb40d172c83
3f19f57e4cb26301321e4b735ff17afb
a12c79e38486790eb02eb90bcd9c249c
18c3430af8d9d451f12894e55e073880
f649f489578cf198f1d5acd3710c5846


In [63]:
decryptor = cipher.decryptor()
decryptor.update(c)

b"Don't forget to respect the block length. A reversible padding scheme is needed in general.#####"

Notice what happens when blocks repeat: 

In [27]:
m = b"Don't use ECB...Don't use ECB...Don't use ECB..."

c = encryptor.update(m)

for i in range(len(c)//16):
    print(c[16*i:16*(i+1)].hex())

b737d676da03d2046d2eafdc349e668a
b737d676da03d2046d2eafdc349e668a
b737d676da03d2046d2eafdc349e668a


This breaks semantic security, since the attacker should not be allowed to know that the message has repeating blocks.

<b>To do</b>: Encrypt the same message using AES in cipher block chaining (CBC) mode "by hand" (<i>i.e.</i>, **don't** use the `cryptography` API except to encrypt single blocks). Verify that Bob is able to decrypt the received ciphertext knowing only the shared secret key $k$.

In [54]:
m = b"Don't use ECB...Don't use ECB...Don't use ECB..."


k = os.urandom(16)  # Clé de 128 bits (16 octets)

# Créez un objet Cipher en utilisant AES en mode CBC
cipher = Cipher(AES(k), ECB(), default_backend()) 

# Créez un encrypteur en utilisant l'objet Cipher
encryptor = cipher.encryptor()

# Chiffrez le message par blocs
ciphertext = b""
for i in range(0, len(m), 16):
    block = m[i:i+16]  # Récupérez le bloc de texte en clair actuel de 16 octets
    encrypted_block = encryptor.update(block)
    ciphertext += encrypted_block

print(ciphertext.hex())

decryptor = cipher.decryptor()
decrypted_text = decryptor.update(ciphertext) 
print("Message décrypté :", decrypted_text)

c2ac1b6ce85fa3c2702c26f9fc9048d0c2ac1b6ce85fa3c2702c26f9fc9048d0c2ac1b6ce85fa3c2702c26f9fc9048d0
Message décrypté : b"Don't use ECB...Don't use ECB...Don't use ECB..."


## 2) Shuffling cards, pt. 1

Let's turn to a seemingly unrelated problem: that of a casino wanting to "impredictably" (from the point of view of the players) but "reproducibly" (from its point of view) shuffle a deck of digital playing cards with a 128-bit shuffle key (probably generated from a master key using a CSPRNG). There are $52! \approx 2^{226}$ different ways to shuffle a deck of cards, so the whole description of a shuffle couldn't possibly fit in the shuffle key: it has to be securely derived from it.

The first, "easiest" way to do it would be to:

1. Encrypt all card values using the shuffle key.
2. Sort lexicographically the resulting ciphertexts.
3. Assign to every "plaincard" its position in the sorted "ciphercards" list.

Hence obtaining a shuffled list of the original cards.

<b>To do</b>: Do it! What is the first poker hand (first 5 cards) dealt from your shuffled card deck?

In [15]:
from cryptography.hazmat.backends import default_backend
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
import os

# Generate a shuffle key
shuffle_key = os.urandom(16)

# Define the card values as strings
card = ['1', '2', '3', '4', '5', '6', '7', '8', '9', '10', 'J', 'Q', 'K']

# Define the suits
suits = ['Hearts', 'Diamonds', 'Clubs', 'Spades']

# Encrypt each card value using the shuffle key for each suit
encrypted_deck = []

for suit in suits:
    for card_value in card:
        cipher = Cipher(algorithms.AES(shuffle_key), modes.ECB(), default_backend())
        encryptor = cipher.encryptor()

        # Pad the card value to 16 bytes (AES block size)
        card_bytes = f'{card_value} of {suit}'.encode('utf-8') + b'\x00' * (16 - len(card_value + " of " + suit))

        encrypted_card = encryptor.update(card_bytes)  # Encrypt the card value
        encrypted_deck.append(encrypted_card)

# Sort the encrypted deck lexicographically
sorted_deck = sorted(encrypted_deck)

# Assign each plaincard its position in the sorted ciphercards list
card_to_position = {}

for i, card_value in enumerate(card):
    for suit in suits:
        plain_card = f'{card_value} of {suit}'
        position = sorted_deck.index(encrypted_deck[i * len(suits) + suits.index(suit)])
        card_to_position[plain_card] = position

# Print the assigned positions for each plaincard
for plain_card, position in card_to_position.items():
    print(f'{plain_card}: {position}')

1 of Hearts: 25
1 of Diamonds: 26
1 of Clubs: 28
1 of Spades: 31
2 of Hearts: 33
2 of Diamonds: 9
2 of Clubs: 32
2 of Spades: 14
3 of Hearts: 34
3 of Diamonds: 0
3 of Clubs: 15
3 of Spades: 7
4 of Hearts: 36
4 of Diamonds: 6
4 of Clubs: 49
4 of Spades: 39
5 of Hearts: 3
5 of Diamonds: 11
5 of Clubs: 43
5 of Spades: 38
6 of Hearts: 13
6 of Diamonds: 5
6 of Clubs: 47
6 of Spades: 51
7 of Hearts: 35
7 of Diamonds: 4
7 of Clubs: 48
7 of Spades: 8
8 of Hearts: 12
8 of Diamonds: 17
8 of Clubs: 16
8 of Spades: 42
9 of Hearts: 24
9 of Diamonds: 22
9 of Clubs: 1
9 of Spades: 10
10 of Hearts: 20
10 of Diamonds: 19
10 of Clubs: 27
10 of Spades: 50
J of Hearts: 44
J of Diamonds: 18
J of Clubs: 30
J of Spades: 29
Q of Hearts: 2
Q of Diamonds: 23
Q of Clubs: 21
Q of Spades: 46
K of Hearts: 37
K of Diamonds: 41
K of Clubs: 40
K of Spades: 45


## 3) Shuffling cards, pt. 2

The above method works well, but rapidly becomes incovenient when the number of objects to permute gets large. Suppose for example that we want to shuffle not 52, but 52000 cards: with the above method, getting the top 5 would require 52000 single AES evaluations. A more efficient solution (look up  "Format-preserving encryption" for more information) is the following:

<ol>
    <li>Write every card value as a 2-byte word.</li>
    <li>Given a card value $m$, apply to it 3 rounds of a Feistel network with inner function $f$, where        
        $ f(b) = $ first byte of the AES encryption of byte $b$ padded with 15 # signs.</li> 
    <li>Repeat step 2. until the resulting value falls in the $[0,51999]$ range; this is the value of the card at position $m$ after the permutation.
</ol>

<b>To do:</b> What are the 5 first cards we get out of this 52000-card deck? How many AES evaluations did this take?

In [13]:
from cryptography.hazmat.primitives.ciphers import Cipher, algorithms, modes
from cryptography.hazmat.backends import default_backend
from cryptography.fernet import Fernet

card_encrypted = []

# Write every card value as a 2-byte word
for card_value in range(52001):
    card_encrypted.append(card_value.to_bytes(2, byteorder='big')) # Print the card value as a 2-byte word

# Given a card value m, apply to it 3 rounds of a Feistel network with inner function f, where f(b) = first byte of the AES encryption of byte b padded with 15 signs.

def f(b):
    # Pad the input byte with 15 null bytes to create a 16-byte block
    block = b + b'\x00'*15
    
    # Create an AES cipher object with a random key
    key = Fernet.generate_key()[:16]
    backend = default_backend()
    cipher = Cipher(algorithms.AES(key), modes.ECB(), backend=backend)
    
    # Encrypt the block using AES and return the first byte of the result
    encryptor = cipher.encryptor()
    encrypted_block = encryptor.update(block) + encryptor.finalize()
    return encrypted_block[0]

print(f(b'\x01'))

b'\x00\x00' b'\x00\x01' b'\x00\x02' b'\x00\x03' b'\x00\x04' b'\x00\x05' b'\x00\x06' b'\x00\x07' b'\x00\x08' b'\x00\t' b'\x00\n' b'\x00\x0b' b'\x00\x0c' b'\x00\r' b'\x00\x0e' b'\x00\x0f' b'\x00\x10' b'\x00\x11' b'\x00\x12' b'\x00\x13' b'\x00\x14' b'\x00\x15' b'\x00\x16' b'\x00\x17' b'\x00\x18' b'\x00\x19' b'\x00\x1a' b'\x00\x1b' b'\x00\x1c' b'\x00\x1d' b'\x00\x1e' b'\x00\x1f' b'\x00 ' b'\x00!' b'\x00"' b'\x00#' b'\x00$' b'\x00%' b'\x00&' b"\x00'" b'\x00(' b'\x00)' b'\x00*' b'\x00+' b'\x00,' b'\x00-' b'\x00.' b'\x00/' b'\x000' b'\x001' b'\x002' b'\x003' b'\x004' b'\x005' b'\x006' b'\x007' b'\x008' b'\x009' b'\x00:' b'\x00;' b'\x00<' b'\x00=' b'\x00>' b'\x00?' b'\x00@' b'\x00A' b'\x00B' b'\x00C' b'\x00D' b'\x00E' b'\x00F' b'\x00G' b'\x00H' b'\x00I' b'\x00J' b'\x00K' b'\x00L' b'\x00M' b'\x00N' b'\x00O' b'\x00P' b'\x00Q' b'\x00R' b'\x00S' b'\x00T' b'\x00U' b'\x00V' b'\x00W' b'\x00X' b'\x00Y' b'\x00Z' b'\x00[' b'\x00\\' b'\x00]' b'\x00^' b'\x00_' b'\x00`' b'\x00a' b'\x00b' b'\x00c' b'\x00d' 